In [73]:
#importing necessery libraries for future analysis of the dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import seaborn as sns
pd.set_option("display.max_rows", None, "display.max_columns", None)
from pprint import pprint
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import pandas as pd
import string
from matplotlib import rcParams
from nltk import WordNetLemmatizer
from wordcloud import WordCloud, STOPWORDS
from nltk.corpus import stopwords
from nltk import pos_tag, sent_tokenize, word_tokenize, BigramAssocMeasures,\
    BigramCollocationFinder, TrigramAssocMeasures, TrigramCollocationFinder
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))
from collections import Counter
#load libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
# Import libraries
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

In [2]:
#using pandas library and 'read_csv' function to read BlackFriday csv file as file already formated for us from Kaggle
airbnb=pd.read_csv("../input/bay-area-airbnb-data-updated-2020/Airbnb_Listings.csv")
#examing head of BlackFriday csv file 
airbnb.head(3)

In [3]:
feature_df = airbnb[['id','listing_url','name']]

In [4]:
feature_df.head()

In [5]:
neighbourhoods = pd.read_csv("../input/bay-area-airbnb-data-updated-2020/neighbourhoods.csv")
neighbourhoods.head(100)

In [6]:
airbnb.describe(include='all')

In [7]:
airbnb['host_tenure_year']= (pd.Timestamp.now() - pd.to_datetime(airbnb['host_since'], errors='coerce')).astype("timedelta64[Y]").astype(int, errors='ignore')
airbnb['host_tenure_year'] = airbnb['host_tenure_year'].fillna(0)
airbnb['host_tenure_year'] .isnull().sum()

In [8]:
feature_df=pd.concat([feature_df, airbnb['host_tenure_year'].fillna(0)], axis=1)

In [9]:
type(airbnb['host_tenure_year'])
feature_df.head()

In [10]:
airbnb['host_tenure_year'].value_counts().sort_values()

In [11]:
airbnb.host_response_time.value_counts()

In [12]:
len(airbnb)

In [13]:
#checking type of every column in the dataset
airbnb.dtypes

In [14]:
#.sort_values(ascending=True)
percent_missing = airbnb.isnull().sum()* 100 /len(airbnb)
missing_value_df = pd.DataFrame({'column_name': airbnb.columns,
                                 'percent_missing': percent_missing}).sort_values(by='percent_missing',ascending =False)
missing_value_df

In [15]:
# find missing values and single value columns & url in data:
#[i for i in airbnb.columns if "url" in i]
del_cols = []

for col in airbnb.columns:
    if airbnb[col].isna().all():
        del_cols.append(col)
    elif airbnb[col].nunique() == 1:
        del_cols.append(col)
    elif "url" in col:
        del_cols.append(col)

del_cols

In [16]:
airbnb = airbnb.drop(del_cols, axis=1)


In [17]:
#airbnb.columns
cat_features = airbnb.select_dtypes(include=['object']).columns
cat_features

In [18]:
cat_cols = []
for col in cat_features:
    list_unique = (col, len(airbnb[col].unique()))
    print(col)
    print(airbnb[col].value_counts(normalize=True).loc[lambda x : x>0.2])
    cat_cols.append(list_unique)
    #print(list_unique)

#output = sorted(cat_cols, key=lambda x: x[-1])
#output



host_response_time
within an hour    0.737532
property_type
House    0.514472

host_response_rate
100%    0.810676

host_acceptance_rate
100%    0.340864

host_is_superhost
f    0.552355
t    0.447645

host_identity_verified
f    0.562465
t    0.437535

room_type
Private room       0.478327
Entire home/apt    0.475142

extra_people
$0.00    0.472788

cancellation_policy
flexible                       0.390112
moderate                       0.315469
strict_14_with_grace_period    0.294281

instant_bookable
f    0.55214
t    0.44786

require_guest_profile_picture
f    0.968148

require_guest_phone_verification
f    0.968425

In [19]:
feature_df['host_response_in_1hr'] = airbnb['host_response_time'].apply(lambda x: int(x=='within an hour') if isinstance(x, str) else 0)
feature_df['host_response_rate100']= airbnb['host_response_rate'].apply(lambda x: int(x=='100%') if isinstance(x, str) else 0)
feature_df['host_acceptance_rate100']=airbnb['host_acceptance_rate'].apply(lambda x: int(x=='100%') if isinstance(x, str) else 0)
feature_df['host_is_superhost'] = airbnb['host_is_superhost'].apply(lambda x: int(x=='t') if isinstance(x, str) else 0)
feature_df['host_identity_verified']= airbnb['host_identity_verified'].apply(lambda x: int(x=='t') if isinstance(x, str) else 0)
feature_df['room_type_private_room']= airbnb['room_type'].apply(lambda x: int(x=='Private room') if isinstance(x, str) else 0)
feature_df['room_type_entire_home']=airbnb['room_type'].apply(lambda x: int(x=='Entire home/apt') if isinstance(x, str) else 0)
feature_df['charge_extra_people_fee']=airbnb['extra_people'].apply(lambda x: int(x!='$0.00') if isinstance(x, str) else 0)

feature_df['cancellation_policy_flexible']= airbnb['cancellation_policy'].apply(lambda x: int(x=='flexible') if isinstance(x, str) else 0)
feature_df['cancellation_policy_moderate']=airbnb['cancellation_policy'].apply(lambda x: int(x=='moderate') if isinstance(x, str) else 0)
feature_df['cancellation_policy_strict']=airbnb['cancellation_policy'].apply(lambda x: int(x=='strict_14_with_grace_period') if isinstance(x, str) else 0)
feature_df['instant_bookable']=airbnb['instant_bookable'].apply(lambda x: int(x=='t') if isinstance(x, str) else 0)




In [20]:
feature_df.head()

In [21]:
numeric_features = airbnb.select_dtypes(include=['int64', 'float64']).columns
airbnb[numeric_features].describe()

# Clean price

In [22]:
## apply function to multiple columns 
def price_cov(x):
    return x.str.replace('$', '').str.replace(',', '').astype(float)

price_colname =['price','security_deposit','cleaning_fee','monthly_price','weekly_price']
airbnb[price_colname] = airbnb[price_colname].apply(price_cov, axis=1)


In [23]:
airbnb[price_colname].head()

In [24]:
## missing value inputation 
## monthly_price weekly_price
price_complete =airbnb[airbnb['monthly_price'].notnull() & airbnb['weekly_price'].notnull()][['monthly_price','weekly_price','price','cleaning_fee','security_deposit']]
price_complete.sort_values(by='price')
# create columes weekly_special_price monthly_special_price 

price_complete['weekly_special_price'] = price_complete['price']*7- price_complete['weekly_price']
price_complete['monthly_special_price'] = price_complete['price']*31- price_complete['monthly_price']

price_complete.fillna(0, inplace=True)



In [25]:
price_complete.head(10)

In [26]:
## append selected price_colname to feature_df
feature_df=pd.concat([feature_df, airbnb[['price','security_deposit','cleaning_fee']].fillna(0)], axis=1)

In [27]:
feature_df=pd.concat([feature_df, price_complete[['weekly_special_price','monthly_special_price']]], axis=1).fillna(0)
feature_df.head()

In [28]:
len(feature_df)

# Clean location

In [29]:
airbnb[['street','neighbourhood','neighbourhood_cleansed','city','state','zipcode','market','smart_location','latitude','longitude','is_location_exact']].head(10)

In [30]:
np.sort(airbnb.neighbourhood_cleansed.unique()).tolist()

In [31]:
airbnb.market.unique().tolist()

In [32]:
neighbourhoods.neighbourhood.tolist()

In [33]:
feature_df['neighbourhood_cleansed']=airbnb.neighbourhood_cleansed
feature_df.head()

# Clean room type

In [34]:
room_colname=['property_type',
'room_type',
'accommodates',
'bathrooms',
'bedrooms',
'beds',
'bed_type',
'amenities',
'square_feet']
airbnb[room_colname].head(10)

In [35]:
numeric_features = airbnb.select_dtypes(
    include=['int64', 'float64', 'float32', 'int32'])
nominal_features = airbnb.select_dtypes(include=['object'])

for item in room_colname:
    print(item)
    print(airbnb[item].unique())

In [36]:
nominal_features = airbnb[room_colname].select_dtypes(include=['object'])

In [37]:
nominal_features.head()

In [38]:
# Cat_features.head()
# pd.get_dummies(data,
#                       columns=[
#                           'host_is_superhost', 'host_identity_verified',
#                           'instant_bookable', 'cancellation_policy',
#                           'room_type'
#                       ],
#                       prefix=[
#                           'Superhost', 'Host verified', 'Instantbook',
#                           'Cancellation', 'Room type'
#                       ],
#                       dtype='int64')

In [39]:
#transform into lower case and export
airbnb['amenities'].str.lower().to_csv("amenities.csv")

# Create Dummy Variable for top 50 amenities

[Where Condition Reference](https://towardsdatascience.com/3-methods-to-create-conditional-columns-with-python-pandas-and-numpy-a6cd4be9da53)


In [40]:
results = Counter()
amenities = airbnb['amenities'].str.lower()
amenities.str.strip('{}')\
               .str.replace('"', '')\
               .str.lstrip('\"')\
               .str.rstrip('\"')\
               .str.split(',')\
               .apply(results.update)
sub_df = pd.DataFrame(results.most_common(50), columns=['amenities', 'count'])
sub_df['ratio'] = round(sub_df['count'] / len(amenities), 4)
sub_df.sort_values(by=['count'], ascending=False)


In [41]:
airbnb['amenities']= airbnb['amenities'].str.lower()
for item in sub_df.amenities.tolist():
    feature_df[item.replace(" ","_")] = airbnb['amenities'].apply(lambda x: int(item in x) if isinstance(x, str) else 0)


In [42]:
#airbnb['amenities']= airbnb['amenities'].str.lower()
#type(airbnb['amenities'][1])
#feature_df["wifi"] = np.where("wifi" in airbnb['amenities'],1, 0)

feature_df["wifi"].sum()
feature_df["wifi"].describe()

In [43]:
# 'tv' in airbnb['amenities'].str.lower()[1]
# for item in sub_df.amenities.tolist():
#     print (item)
#     print (item in airbnb['amenities'][1])

In [44]:
feature_df.columns.tolist()

## Visualization


In [45]:
###pairplot##
sns.set(color_codes=True)
sns.set(style="white", palette="muted")
df = feature_df.drop('id',axis = 1)
df = df.drop('listing_url',axis = 1)
fig = plt.figure(figsize = [20,15])
ax = fig.gca()
df.hist(ax = ax, bins = 50)
plt.show()

In [46]:
df_corr = df._get_numeric_data().astype(float).corr()

# f,ax = plt.subplots(figsize= (20,20))
# sns.heatmap(df.corr(), annot = True, linewidths=0.5, fmt = '.1f', ax=ax, cmap = 'winter')
# plt.show()
_, ax = plt.subplots(figsize=(25, 25))

sns.heatmap(df_corr,
            square=True,
            cmap='RdBu_r',
            linewidths=.5,
            annot=True,
            fmt='.2f',ax=ax).figure.tight_layout()

plt.show()


In [47]:
# filter the correlation matrix with threshold

filtered_df = df_corr[((df_corr >= .5) | (df_corr <= -.5)) & (df_corr !=1)].dropna(how="all")

_, ax = plt.subplots(figsize = (20,20))

sns.heatmap(filtered_df,
            square=True,
            cmap='RdBu_r',
            linewidths=.5,
            annot=True,
            fmt='.2f',
            ax=ax)

plt.show()

# Text Normalization
Ref: [code](https://www.kaggle.com/peterwittek/wordclouds-of-titles-and-abstracts-with-n-grams)


In [48]:
def get_bitrigrams(full_text, threshold=30):
    if isinstance(full_text, str):
        text = full_text
    else:
        text = " ".join(full_text)
    bigram_measures = BigramAssocMeasures()
    trigram_measures = TrigramAssocMeasures()
    finder = BigramCollocationFinder.from_words(text.split())
    finder.apply_freq_filter(3)
    bigrams = {" ".join(words): "_".join(words)
               for words in finder.above_score(bigram_measures.likelihood_ratio, threshold)}
    finder = TrigramCollocationFinder.from_words(text.split())
    finder.apply_freq_filter(3)
    trigrams = {" ".join(words): "_".join(words)
                for words in finder.above_score(trigram_measures.likelihood_ratio, threshold)}
    return bigrams, trigrams


def replace_bitrigrams(text, bigrams, trigrams):
    if isinstance(text, str):
        texts = [text]
    else:
        texts = text
    new_texts = []
    for t in texts:
        t_new = t
        for k, v in trigrams.items():
            t_new = t_new.replace(k, v)
        for k, v in bigrams.items():
            t_new = t_new.replace(" " + k + " ", " " + v + " ")
        new_texts.append(t_new)
    if len(new_texts) == 1:
        return new_texts[0]
    else:
        return new_texts

    
def process_text(text, lemmatizer, translate_table, stopwords):
    processed_text = ""
    for sentence in sent_tokenize(text):
        tagged_sentence = pos_tag(word_tokenize(sentence.translate(translate_table)))
        for word, tag in tagged_sentence:
            word = word.lower()
            if word not in stopwords:
                if tag[0] != 'V':
                    processed_text += lemmatizer.lemmatize(word) + " "
    return processed_text


def get_all_processed_texts(texts, lemmatizer, translate_table, stopwords):
    processed_texts = []
    for index, doc in enumerate(texts):
        processed_texts.append(process_text(doc, wordnet_lemmatizer, translate_table, stop))
    bigrams, trigrams = get_bitrigrams(processed_texts)
    very_processed_texts = replace_bitrigrams(processed_texts, bigrams, trigrams)
    return " ".join(very_processed_texts)

In [49]:
# wordcloud = WordCloud(background_color="white").\
#     generate(get_all_processed_texts(records, wordnet_lemmatizer))
# plt.figure(figsize=(15, 10))
# plt.axis("off")
# plt.imshow(wordcloud)

In [50]:
# records = airbnb['amenities'].str.lower()
# wordnet_lemmatizer = WordNetLemmatizer()
# stop = set(stopwords.words('english'))
# translate_table = dict((ord(char), " ") for char in string.punctuation)

In [51]:
# amenitieslist =[]
# for itemlist in records:
#     amenitieslist.extend(itemlist.replace('"',' ').replace('{','').replace('}','').split(','))
# amenitiesdic = WordCloud().process_text(' '.join(amenitieslist))

# sorted([(value,key) for (key,value) in amenitiesdic.items()], reverse=True)


[(6794, 'smoke alarm'),
 (6214, 'essentials shampoo'),
 (6178, 'monoxide alarm'),
 (6178, 'carbon monoxide'),
 (6153, 'alarm carbon'),
 (5998, 'laptop friendly'),
 (5998, 'friendly workspace'),
 (5997, 'hair dryer'),
 (5768, 'washer dryer'),
 (5739, 'hangers hair'),
 (5537, 'dryer smoke'),
 (5344, 'dryer iron'),
 (5311, 'hot water'),
 (5277, 'free parking'),
 (5121, 'iron laptop'),
 (5083, 'kitchen free'),
 (4571, 'air conditioning'),
 (4512, 'wifi air'),
 (4285, 'heating washer'),
 (4043, 'street parking'),
 (4043, 'free street'),
 (4040, 'fire extinguisher'),
 (3913, 'extinguisher essentials'),
 (3907, 'coffee maker'),
 (3752, 'microwave coffee'),
 (3749, 'maker refrigerator'),
 (3710, 'bed linens'),
 (3677, 'bedroom door'),
 (3668, 'first aid'),
 (3668, 'aid kit'),
 (3642, 'alarm first'),
 (3623, 'tv wifi'),
 (3447, 'cooking basics'),
 (3433, 'water bed'),
 (3357, 'silverware cooking'),
 (3308, 'shampoo lock'),
 (3257, 'self check'),
 (3201, 'oven stove'),
 (3180, 'door hangers'),
 (3083, 'basics oven'),
 (3052, 'private entrance'),

In [52]:
# numeric_features = data.select_dtypes(
#     include=['int64', 'float64', 'float32', 'int32'])
# nominal_features = data.select_dtypes(include=['object'])

![](http://)

In [53]:
# #loop and get the unique value for each object column 
# display(HTML(airbnb[room_colname].select_dtypes(include=['object']).T.apply(lambda x: x.unique(), axis=1).to_frame().to_html()))


# Clean Review 
Last Review to date

In [54]:

feature_df=pd.concat([feature_df, airbnb.loc[:, airbnb.columns.str.startswith("review")].fillna(0)], axis=1)

In [64]:
feature_df=feature_df.head()

In [56]:
feature_df.to_csv('/cleaned_dataset.csv')

## Feature Selection

In [57]:
feature_df.select_dtypes(include=['object']).columns

In [58]:
feature_df["neighbourhood_cleansed"].head()

In [67]:
feature_df2= pd.get_dummies(feature_df, columns=["neighbourhood_cleansed"], prefix=["neighbourhood_is"])

In [78]:
# Declare feature vector and target variable
X = feature_df2.drop(['id','price','listing_url', 'name'], axis = 1)
y = feature_df2['price']

In [79]:
# Normalize feature vector
X2 = StandardScaler().fit_transform(X)

In [80]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size = 0.30, random_state = 0)

In [81]:
# instantiate the classifier with n_estimators = 100
clf = RandomForestClassifier(n_estimators=100, random_state=0)

In [82]:
# fit the classifier to the training set
clf.fit(X_train, y_train)

In [83]:
# predict on the test set
y_pred = clf.predict(X_test)

In [87]:
# visualize feature importance

plt.figure(num=None, figsize=(10,8), dpi=80, facecolor='w', edgecolor='k')

feat_importances = pd.Series(clf.feature_importances_, index= X.columns)

feat_importances.nlargest(15).sort_values(ascending=True).plot(kind='barh')
